In [28]:
from pyspark.sql.functions import from_utc_timestamp
from pyspark.sql.functions import regexp_replace
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

df = sqlc.read.format('com.databricks.spark.csv').\
options(header='true', inferschema='true').\
load('file:///home/oxclo/datafiles/wind/*')

df = df.withColumn('Interval_End_Time',\
                   from_utc_timestamp(regexp_replace(df.Interval_End_Time, '(.*)\\? (.*)', '$1 $2'), 'utc'))

df = df.withColumnRenamed('Station_ID', 'stationid')\
.withColumnRenamed('Interval_End_Time', 'time')\
.withColumnRenamed('Wind_Direction_Deg', 'direction')\
.withColumnRenamed('Ambient_Temperature_Deg_C', 'temp')\
.withColumnRenamed('Wind_Velocity_Mtr_Sec', 'velocity')

df.select(df.stationid, df.time, df.direction, df.temp, df.velocity)\
.write.format("org.apache.spark.sql.cassandra").mode('append').options(table="winddata", keyspace="wind").save()
